Imports

In [1]:
import pandas as pd
from Bio import SeqIO
from ushuffle import shuffle, Shuffler
import torch

Constants

In [2]:
NUM_PROTEINS = 20577
K_MER_SIZE = 1

Functions

In [3]:
def convert_fasta_to_dataframe(fasta_file_path, num_proteins):
    lengths = [None for _ in range(num_proteins)]
    sequences = [None for _ in range(num_proteins)]
    i = 0
    for record in SeqIO.parse(fasta_file_path, "fasta"):
        lengths[i] = len(record.seq)
        sequences[i] = str(record.seq)
        i += 1
    df_dic = {'Sequence': sequences, 'length': lengths}
    df = pd.DataFrame(df_dic)
    df = df.drop_duplicates(subset='Sequence').reset_index(drop=True)
    return df.loc[df['length'] <= 512].loc[df['length'] >= 20].reset_index(drop=True)

def shuffle_sequences(df_proteome, k_let):
    seqs_ushuffled = [None for _ in range(len(df_proteome))]
    lengths_ushuffled = [None for _ in range(len(df_proteome))]
    seqs_not_shuff_inds = []
    for i in range(len(df_proteome)):
        if k_let >= 1:
            orig_seq = df_proteome.iloc[i, 0]
            orig_seq_bytes = orig_seq.encode()
            shuffler = Shuffler(orig_seq_bytes, k_let)
            shuff_seq_bytes = shuffler.shuffle()
            shuff_seq = shuff_seq_bytes.decode()
            if orig_seq==shuff_seq:
                seqs_not_shuff_inds.append(i)

        if k_let == 1:
            orig_seq = df_proteome.iloc[i, 0]
            orig_seq_bytes = orig_seq[1:].encode()
            shuffler = Shuffler(orig_seq_bytes, k_let)
            shuff_seq_bytes = shuffler.shuffle()
            shuff_seq = orig_seq[0] + shuff_seq_bytes.decode()

        seqs_ushuffled[i] = shuff_seq
        lengths_ushuffled[i] = len(shuff_seq)

    df_dic = {'Sequence': seqs_ushuffled, 'length': lengths_ushuffled}
    return pd.DataFrame(df_dic), seqs_not_shuff_inds

def convert_dataframe_to_fasta(fasta_file_path, df_real_proteins, df_shuffled_sequences):
    fasta_file = open(fasta_file_path, 'w')

    for i in range(len(df_real_proteins)):
        fasta_file.write(f'>n_{i}\n')
        fasta_file.write(f'{df_real_proteins.iloc[i, 0]}\n')
        fasta_file.write(f'>s_{i}\n')
        fasta_file.write(f'{df_shuffled_sequences.iloc[i, 0]}\n')

    fasta_file.close()
    
def extarct_natural_and_real_sequences(fasta_file_path):
    natural_lengths = {}
    natural_sequences = {}
    shuffled_sequences = {}
    natural_sequences_list = []
    shuffled_sequences_list = []
    lengths = []
    for record in SeqIO.parse(fasta_file_path, "fasta"):
        rec_id = record.id.split('_')
        if rec_id[0] == 'n':
            natural_sequences[rec_id[1]] = str(record.seq)
            natural_lengths[rec_id[1]] = len(record.seq)
        else:
            shuffled_sequences[rec_id[1]] = str(record.seq)
    
    for natural_record in natural_sequences:
        if natural_record in shuffled_sequences:
            natural_sequences_list.append(natural_sequences[natural_record])
            shuffled_sequences_list.append(shuffled_sequences[natural_record])
            lengths.append(natural_lengths[natural_record])
    
    all_seqs = natural_sequences_list + shuffled_sequences_list
    human_proteome_vs_ushuffled_dic = {'Seq': all_seqs, 'length': lengths*2}
    return pd.DataFrame(human_proteome_vs_ushuffled_dic)
    
def add_labels(df_dataset):
    half_dataset_size = df_dataset.shape[0]//2
    real_labels = torch.ones(half_dataset_size, dtype=torch.int)
    ushuffled_labels = torch.zeros(half_dataset_size, dtype=torch.int)
    y_data = torch.cat((real_labels, ushuffled_labels))
    df_dataset['label'] = y_data
    return df_dataset

def split_dataset(df_all_sequences):
    mid = len(df_all_sequences)//2
    df_natural = df_all_sequences[:mid].reset_index(drop=True)
    df_shuff = df_all_sequences[mid:].reset_index(drop=True)
    
    eighty_percent = round((0.8*len(df_natural))/10)*10
    df_natural_train = df_natural.sample(eighty_percent)
    train_inds = df_natural_train.index
    
    df_natural_train = df_natural_train.reset_index(drop=True)
    df_natural_test = df_natural.drop(index=train_inds).reset_index(drop=True)
    df_shuff_train = df_shuff.iloc[train_inds,:].reset_index(drop=True)
    df_shuff_test = df_shuff.drop(index=train_inds).reset_index(drop=True)
    
    df_training_set = df_natural_train.append(df_shuff_train).reset_index(drop=True)
    df_test_set = df_natural_test.append(df_shuff_test).reset_index(drop=True)
    
    return df_training_set, df_test_set

Get the human proteome from UniProt

In [4]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz
!gzip -dk UP000005640_9606.fasta.gz

--2022-04-14 16:09:51--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000005640/UP000005640_9606.fasta.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7724727 (7.4M) [application/x-gzip]
Saving to: ‘UP000005640_9606.fasta.gz’

UP000005640_9606.fa 100%[===================>]   7.37M  5.74MB/s    in 1.3s    

2022-04-14 16:09:53 (5.74 MB/s) - ‘UP000005640_9606.fasta.gz’ saved [7724727/7724727]



In [5]:
human_proteome = convert_fasta_to_dataframe('UP000005640_9606.fasta', NUM_PROTEINS)
human_proteome_shuffled, seqs_not_shuffled = shuffle_sequences(human_proteome, K_MER_SIZE)
if len(seqs_not_shuffled) > 0:
    human_proteome = human_proteome.drop(index=seqs_not_shuffled).reset_index(drop=True)
    human_proteome_shuffled = human_proteome_shuffled.drop(index=seqs_not_shuffled).reset_index(drop=True)
human_proteome_vs_shuffled = human_proteome.append(human_proteome_shuffled).reset_index(drop=True)
convert_dataframe_to_fasta('human_proteome_vs_ushuffled.fasta', human_proteome_shuffled, human_proteome_vs_shuffled)
display(human_proteome_vs_shuffled)

,Sequence,length
0,MAWTPLFLFLLTCCPGSNSQAVVTQEPSLTVSPGGTVTLTCGSSTG...,117
1,MGCCGCGGCGGCGGGCGGGCGSCTTCRCYRVGCCSSCCPCCRGCCG...,85
2,MSTFPVLAEDIPLRERHVKGRVDPHFRAPKMEMFQRLLLLLLLSMG...,187
3,MSSTLPALLCVGLCLSQRISAQQQTLPKPFIWAEPHFMVPKEKQVT...,304
4,MNLQRYWGEIPISSSQTNRSSFDLLPREFRLVEVHDPPLHQPSANK...,414
...,...,...
25401,MKAWDRPHLSECTESHLMHCAASYSSAGWMCGESESGLLEEELALG...,448
25402,MVARMRDRQRVRLPLLSRYVSLHQMPLQDGLNTIQFLQTDLRGPTE...,265
25403,MIGESGRNIKSSAEHQKLPISSGVGGFIQLVQFQCLQEVAISLPAG...,360
25404,MPEDEGYSDCTGYSGPWATIHFAEENRDAVQVFLRYEKSGEDRKDE...,174


Perform h-CD-HIT using _human_proteome_vs_ushuffled.fasta_ file with pairwise identity cutoff of 0.9, 0.5 and 0.1 \
(available at http://weizhong-lab.ucsd.edu/cdhit-web-server/cgi-bin/index.cgi?cmd=h-cd-hit)

Count number of natural and shuffled sequences after CD-HIT

In [6]:
cd_hit_num_natural_seqs = !grep -c '>n' cd_hit_human_proteome_vs_ushuffled.fasta # filename after h-CD-HIT
cd_hit_num_natural_seqs = int(cd_hit_num_natural_seqs[0])
print(f' There are {cd_hit_num_natural_seqs} natural sequences after CD-HIT')
cd_hit_num_shuffled_seqs = !grep -c '>s' cd_hit_human_proteome_vs_ushuffled.fasta
cd_hit_num_shuffled_seqs = int(cd_hit_num_shuffled_seqs[0])
print(f' There are {cd_hit_num_shuffled_seqs} shuffled sequences after CD-HIT')

 There are 5857 natural sequences after CD-HIT
 There are 12594 shuffled sequences after CD-HIT


In [7]:
cd_hit_human_proteome_vs_shuffled = extarct_natural_and_real_sequences('cd_hit_human_proteome_vs_ushuffled.fasta')
cd_hit_human_proteome_vs_shuffled = add_labels(cd_hit_human_proteome_vs_shuffled)
display(cd_hit_human_proteome_vs_shuffled)

,Seq,length,label
0,MSTFPVLAEDIPLRERHVKGRVDPHFRAPKMEMFQRLLLLLLLSMG...,187,1
1,MNLQRYWGEIPISSSQTNRSSFDLLPREFRLVEVHDPPLHQPSANK...,414,1
2,MSCHNCSDPQVLCSSGQLFLQPLWDHLRSWEALLQSPFFPVIFSIT...,272,1
3,MRLPAQLLGLLMLWVPGSSEDIVMTQTPLSLPVTPGEPASISCRSS...,121,1
4,MNLAISIALLLTVLQVSRGQKVTSLTACLVDQSLRLDCRHENTSSS...,161,1
...,...,...,...
11693,MELGAERTPKDGSSEMSQELGIGGNKGDELNIIMEDEKTEMSYACR...,253,0
11694,MHIKRNNVFNSKTSQCAIGYFLSRVDIQASDQFTINLRTTNPFVWP...,453,0
11695,MRGSKMGERTFDTKSGCAALIGPKAPDPPLERSEEVVYSGLEEACP...,448,0
11696,MQLFALELHRLRTHAHRLEHGPDVWRRLYLDDKVLWDVRWPLRGAR...,265,0


Split dataset to 80% training set and 20% test set

In [8]:
df_train, df_test = split_dataset(cd_hit_human_proteome_vs_shuffled)
df_train.to_csv('training_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)
display(df_train)
display(df_test)

,Seq,length,label
0,MELATRYQIPKEVADIFNAPSDDEEFVGFRDDVPMETLSSEESCDS...,454,1
1,MAPSRLQLGLRAAYSGISSVAGFSIFLVWTVVYRQPGTAAMGGLAG...,146,1
2,MQKSEGSGGTQLKNRATGNYDQRTSSSTQLKHRNAVQGSKSSLSTS...,248,1
3,MNGLPSAEAPGGAGCALAGLPPLPRGLSGLLNASGGSWRELERVYS...,189,1
4,MGRPLLLPLLPLLLPPAFLQPSGSTGSGPSYLYGVTQPKHLSASMG...,303,1
...,...,...,...
9355,MQQLANKGQDDLDSTIKWKPNASHYWIEAAPDSNQLLGLMRVNSNL...,433,0
9356,MLASHFLYTTNLKRAINESLGEGKNAETEQAIETIFVDTRHPKHEL...,268,0
9357,MNVVAKLSVADDYFNHRIGLYLNINKYQASSLSTMLVRDQVYTSIA...,350,0
9358,MERESQYEGRSETLIQLVAQEESKFEYVKLKPQLGKGLQEEISQSP...,190,0


,Seq,length,label
0,MSCHNCSDPQVLCSSGQLFLQPLWDHLRSWEALLQSPFFPVIFSIT...,272,1
1,MEDRAGEQEQERHSLRLEKLQHWARHRQSGHLLVLAVSQLWLAVVV...,217,1
2,MAPLCPSPWLPLLIPAPAPGLTVQLLLSLLLLVPVHPQRLPRMQED...,459,1
3,MSPLECSECFGDQLLHRTYTWQLTLHSRPNYTRKRDTRSESLEIPI...,448,1
4,MGTPASVVSEPPPWQAPIEARGRKQASANIFQDAELLQIQALFQRS...,147,1
...,...,...,...
2333,MPCRSSPAGQRPTGGRWAGSGQQADGPAPGIFLLGRTLRPQASLGA...,163,0
2334,MKSANKEAQLKLNSMVTLLQAAELQNNNKSPLSKIQRGVEKEPAKK...,500,0
2335,MLYNSQIYNIVKRSYTSFKDTIMIPTEGIGGWTKLNNPLQMNIFQM...,229,0
2336,MEMFSFFVTHVKTSCSAQQKSFRTPGKFRLLMGHDTQFATCKDHQG...,476,0
